In [3]:
from transformers import AutoTokenizer, Phi3ForCausalLM
import torch
checkpoint = "microsoft/Phi-3.5-mini-instruct"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
model = Phi3ForCausalLM.from_pretrained(
    checkpoint,
    torch_dtype=torch.bfloat16,  # Use bfloat16 for memory efficiency
    trust_remote_code=True,
    device_map="auto"  # Automatically map the model to available GPUs
)

prompt = "This is an example script ."
inputs = tokenizer(prompt, return_tensors="pt")

# Generate
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\ASUS\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  6.14it/s]
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
You are not running the flash-attention implementation, expect numerical differences.


'This is an example script .\n\n#!/bin/bash\n\n# This script will create a new user with a home directory and a custom'

In [4]:
from datasets import load_dataset

In [5]:

# 1. Memuat Dataset
dataset = load_dataset("SEACrowd/liputan6")
print("Dataset loaded:", dataset)


Dataset loaded: DatasetDict({
    train: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 193883
    })
    test: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 10972
    })
    validation: Dataset({
        features: ['document', 'id', 'summary'],
        num_rows: 10972
    })
})


In [6]:

# Mengambil sampel artikel dari dataset
sample_article = dataset["train"][0]["document"]
sample_summary = dataset["train"][0]["summary"]
sample_article1 = dataset["train"][1]["document"]
sample_summary1 = dataset["train"][1]["summary"]



In [17]:
# ini one shot
# Artikel baru yang akan diringkas
new_article = dataset["test"][1]["document"]

# 3. Membuat One-Shot Prompt
prompt = f"""
Below is an example of summarization:
Article: {sample_article}
Summary: {sample_summary}

Now, summarize the following article:
Article: {new_article}
Summary:"""

# 4. Tokenisasi dan Inference
inputs = tokenizer(prompt, return_tensors="pt")
generate_ids = model.generate(inputs.input_ids, max_new_tokens=200, num_beams=5, early_stopping=True)
generated_summary = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

# 5. Menampilkan Hasil
print("New Article:\n", new_article)
print("\nGenerated Summary:\n", generated_summary)


New Article:
 Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak m

In [5]:
# ini few shot
# Artikel baru yang akan diringkas
new_article = dataset["test"][1]["document"]

# 3. Membuat One-Shot Prompt
prompt = f"""
Below is an example of summarization:
Article: {sample_article}
Summary: {sample_summary}
Article: {sample_article1}
Summary: {sample_summary1}

Now, summarize the following article:
Article: {new_article}
Summary:"""

# 4. Tokenisasi dan Inference
inputs = tokenizer(prompt, return_tensors="pt")
generate_ids = model.generate(inputs.input_ids, max_new_tokens=200, num_beams=5, early_stopping=True)
generated_summary_few_shot = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

# 5. Menampilkan Hasil
print("New Article:\n", new_article)
print("\nGenerated Summary:\n", generated_summary_few_shot)


New Article:
 Liputan6 . com , Bandar Lampung : Sebanyak 51 anak di bawah umur lima tahun terserang busung lapar atau marasmus karena kekurangan gizi di Kota Madya Bandar Lampung . Lima di antaranya tewas . Data tersebut diungkapkan Kepala Dinas Kesehatan Kota Bandar Lampung M . Sudarman , baru-baru ini . Menurut Sudarman , Dinas Kesehatan Bandar Lampung mencatat sekitar 51 anak terserang busung lapar yang tersebar di beberapa kecamatan , selama periode 1999 sampai 2001 . Kebanyakan anak penderita busung tersebut berasal dari keluarga yang hidup di bawah garis kemiskinan . Selain kekurangan gizi , komplikasi radang paru-paru juga menjadi satu faktor penyebab kematian anak penderita busung lapar tersebut . Data Dinas Kesehatan menunjukkan pada 1999 , ditemukan 41 anak terserang penyakit busung lapar . Sebagian besar penderita berdomisili di kampung miskin Umbul Kunci . Jumlah penderita busung lapar menurun pada 2000 , yakni hanya sembilan anak . Sedangkan September 2001 , seorang anak m

In [8]:
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [23]:
# 5. Menghitung BLEU Score
reference_summary = dataset["test"][1]["summary"]
def calculate_bleu_score(reference, candidate):
    # Tokenisasi menggunakan nltk
    reference_tokens = nltk.word_tokenize(reference.lower())
    candidate_tokens = nltk.word_tokenize(candidate.lower())
    # Menggunakan smoothing untuk menangani n-gram yang hilang
    smoothing = SmoothingFunction().method4
    return sentence_bleu([reference_tokens], candidate_tokens, smoothing_function=smoothing)

bleu_score = calculate_bleu_score(reference_summary, generated_summary)
# bleu_score_few_shot = calculate_bleu_score(reference_summary, generated_summary_few_shot)

In [24]:
print(f"\nBLEU Score one shot: {bleu_score:.4f}")
# print(f"\nBLEU Score few shot: {bleu_score_few_shot:.4f}")


BLEU Score one shot: 0.0210


In [13]:
==================================================================================================================================

In [19]:
import random
test_samples = 10
new_article = dataset["test"][1]["document"]
# 5. Menghitung BLEU Score untuk 100 Sampel
bleu_scores = []
sample_results = []

for idx in range(test_samples):
    # Ambil artikel dan ringkasan referensi
    article = dataset["test"][idx]["document"]
    reference_summary = dataset["test"][idx]["summary"]

    # Membuat prompt untuk one-shot
    prompt = f"""
            Below is an example of summarization:
            Article: {sample_article}
            Summary: {sample_summary}

            Now, summarize the following article:
            Article: {new_article}
            Summary:"""

    # Tokenisasi dan inference
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=1024)
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=500, num_beams=5, early_stopping=True)
    generated_summary = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]


In [28]:

    # Menghitung BLEU score
    bleu_score = calculate_bleu_score(reference_summary, generated_summary)

    # Simpan hasil
    bleu_scores.append(bleu_score)
    sample_results.append({
        "article": article,
        "reference_summary": reference_summary,
        "generated_summary": generated_summary,
        "bleu_score": bleu_score
    })


In [33]:
# Menampilkan rata-rata BLEU Score
print(f"\nAverage BLEU Score for 10 samples: {bleu_score:.4f}\n")




Average BLEU Score for 10 samples: 0.0210



In [30]:
import random
test_samples = 10
# 5. Menghitung BLEU Score untuk 100 Sampel
bleu_scores_few_shot = []
sample_results = []

for idx in range(test_samples):
    # Ambil artikel dan ringkasan referensi
    article = dataset["test"][idx]["document"]
    reference_summary = dataset["test"][idx]["summary"]

    # Membuat prompt untuk one-shot
    prompt = f"""
            Below is an example of summarization:
            Article: {sample_article}
            Summary: {sample_summary}
            Article: {sample_article1}
            Summary: {sample_summary1}

            Now, summarize the following article:
            Article: {new_article}
            Summary:"""

    # Tokenisasi dan inference
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, padding=True, max_length=1024)
    generate_ids = model.generate(inputs.input_ids, max_new_tokens=500, num_beams=5, early_stopping=True)
    generated_summary_few_shot = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

    # Menghitung BLEU score
    bleu_score_few_shot = calculate_bleu_score(reference_summary, generated_summary_few_shot)

    # Simpan hasil
    bleu_scores_few_shot.append(bleu_score_few_shot)
    sample_results.append({
        "article": article,
        "reference_summary": reference_summary,
        "generated_summary_few_shot": generated_summary_few_shot,
        "bleu_score": bleu_score
    })
    


In [32]:
# Menampilkan rata-rata BLEU Score
print(f"\nAverage BLEU Score for 10 samples: {bleu_score_few_shot:.4f}\n")




Average BLEU Score for 10 samples: 0.0037

